Introduction : Ce notebook constitue ma participation à la compétition kaggle "Spaceship Titanic". 

L'objectif de l'étude est d'utiliser et d'optimiser un algorithme de Machine Learning de classification pour prévoir si un passager à survécu ou non. 
Deux datasets sont à disposition. Le premier, train.csv, comprend la variable cible à prédire "Transported" pour l'entrainement. Le second, test.csv est utilisé pour réaliser les prédictions. Le fichier sample_submission.csv est le fichier qui servira pour stocker les prédictions et les envoyer. 

La métrique de ce concours est l'accuracy. L'objectif principal est donc de modéliser un modèle prédictif de Machine Learning maximisant l'accuracy. 

In [ ]:
#Import des packages nécessaires 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import model_selection
from sklearn import neighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc, classification_report

sns.set_theme(style='whitegrid', palette='pastel')

In [ ]:
#Import des DataFrames 

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

#Attention : toutes les modifications sur les df devront être faites dans les deux dataframes pour que l'algo de ML soit fonctionnel 

In [ ]:
df_train.head()

In [ ]:
df_test.head()

# I) Nettoyage des données 

Pour cet analyse mettant en oeuvre des algorithmes de Machine Learning, le nettoyage et la préparation des données est une étape absolument décisive. Des hypothèses on été posées et seules celles retenues figurent dans ce notebook. 

## A) Inspection des données 

La première chose à faire avant d'explorer un jeu de données est de s'intéresser aux métadatas s'il y en a et aux informations qu'on a sur les données.
En premier lieu, d'où viennent ces données ? Comment ces données ont-elles été collectées ? Quels types de fichiers a-t-on ? De quelles tailles ? Quelles sont les caractéristiques présentes ?

In [ ]:
#DataFrame train 
df_train.info()
#Il y a 8693 entrées et 14 colonnes. 
#On observe qu'il y a de nombreux NaNs dans les colonnes constituant le df. 
#Nous n'avons pas de fichier metadata à notre disposition

In [ ]:
#DataFrame test
df_test.info()
#Il y a 4277 entrées et 13 colonnes. Nous n'avons pas ici la colonne target (Transported)
#On observe qu'il y a de nombreux NaNs dans les colonnes constituant le df. 
#Nous n'avons pas de fichier metadata à notre disposition.

## B) Vérification de la cohérence et de l'uniformité des données

### 1) Cohérence des données
Tout d'abord, il convient de vérifier que chaque variable est au bon type. L'objectif est d'avoir une uniformité des types de valeurs entre les 2 DataFrames.

In [ ]:
#DataFrame train


#PassengerId doit être en deux colonnes (PassengerGroup et PassengerGroupNumber) et en int
df_train[['PassengerGroup','PassengerGroupNumber']]=df_train['PassengerId'].str.split('_', expand=True).astype(int)

#CryoSleep doit être en booléen
df_train.CryoSleep.unique() #A au moins un NaN, on refera une passe dessus après la gestion des valeurs manquantes

#Cabin a mettre en 3 colonnes et en object / int / object
df_train[['Deck','Num','Side']]=df_train['Cabin'].str.split('/', expand=True) #Contient des NaNs donc sera traité après la gestion des valeurs manquantes

#VIP a mettre en booléen ATTENTION BIEN REGARDER AVEC UNIQUE SI PREND BIEN 2 TRUCS
df_train['VIP'].unique() #Contient des NaNs donc sera traité après la gestion des valeurs manquantes

#Les types des variables suivantes seront traités après le traitement des valeurs manquantes
#Food court a mettre en int
#Shopping mall a mettre en int
#Spa a mettre en int 
#Age a mettre en int
#Room service a mettre en int (vérifier l'explication sur le site)

df_train.info()
df_train.tail()


In [ ]:
#Suppression des colonnes inutiles 
df_train=df_train.drop(['PassengerId','Cabin','Name'], axis = 1)
df_train.head()

In [ ]:
#DataFrame test

#PassengerId doit être en deux colonnes (PassengerGroup et PassengerGroupNumber) et en int
df_test[['PassengerGroup','PassengerGroupNumber']]=df_test['PassengerId'].str.split('_', expand=True).astype(int)

#CryoSleep doit être en booléen
df_test.CryoSleep.unique() #A au moins un NaN, on refera une passe dessus après la gestion des valeurs manquantes

#Cabin a mettre en 3 colonnes et en object / int / object
df_test[['Deck','Num','Side']]=df_test['Cabin'].str.split('/', expand=True) #Contient des NaNs donc sera traité après la gestion des valeurs manquantes

#VIP a mettre en booléen ATTENTION BIEN REGARDER AVEC UNIQUE SI PREND BIEN 2 TRUCS
df_test['VIP'].unique() #Contient des NaNs donc sera traité après la gestion des valeurs manquantes

#Les types des variables suivantes seront traités après le traitement des valeurs manquantes
#Food court a mettre en int
#Shopping mall a mettre en int
#Spa a mettre en int 
#Age a mettre en int
#Room service a mettre en int (vérifier l'explication sur le site)

df_test.info()
df_test.tail()


In [ ]:
#Suppression des colonnes inutiles 
df_test=df_test.drop(['PassengerId','Cabin','Name'], axis = 1)
df_test.head()

### 2) Uniformité des données

Il convient de vérifier que les variables qualitatives sont uniformes, c'est à dire qu'elles doivent être écrites toujours de la même manière en respectant une règle unique

In [ ]:
#DataFrame Train 
#Les variables qualitatives sont : HomePlanet, Destination, Age, VIP, Transported, PassengerGroupNumber, Deck, Side 
df_train['HomePlanet'].unique() #Toutes les planètes sont écrites au même format

df_train['Destination'].unique() #Toutes les destinations sont écrites au même format
#Il est décidé de simplifier les noms et de les mettre en majuscule 
df_train['Destination']=df_train['Destination'].replace('TRAPPIST-1e','TRAPPIST')
df_train['Destination']=df_train['Destination'].replace('55 Cancri e','CANCRI')
df_train['Destination']=df_train['Destination'].replace('PSO J318.5-22','PSO')
df_train['Destination'].unique() 

df_train['Age'].unique() #Tous les ages sont au même format. Après le traitement des NaNs ils seront mis en int

df_train['VIP'].unique() #Tous les VIP sont au même format et pourront donc être mis en booléen après traitement des NaNs

df_train['Transported'].unique() #Etant donné qu'il s'agit d'un booléen ce n'est pas étonnant 

df_train['PassengerGroupNumber'].unique() #Tous les numéros de groupes sont écrits au bon format 

df_train['Deck'].unique() #Tous les decks sont écrits au même format 

df_train['Side'].unique() #Tous les sides sont écrits au même format 

In [ ]:
#DataFrame Test
#Les variables qualitatives sont : HomePlanet, Destination, Age, VIP, Transported, PassengerGroupNumber, Deck, Side 
df_test['HomePlanet'].unique() #Toutes les planètes sont écrites au même format

df_test['Destination'].unique() #Toutes les destinations sont écrites au même format
#Il est décidé de simplifier les noms et de les mettre en majuscule 
df_test['Destination']=df_test['Destination'].replace('TRAPPIST-1e','TRAPPIST')
df_test['Destination']=df_test['Destination'].replace('55 Cancri e','CANCRI')
df_test['Destination']=df_test['Destination'].replace('PSO J318.5-22','PSO')
df_test['Destination'].unique()

df_test['Age'].unique() #Tous les ages sont au même format. Après le traitement des NaNs ils seront mis en int

df_test['VIP'].unique() #Tous les VIP sont au même format et pourront donc être mis en booléen après traitement des NaNs

df_test['PassengerGroupNumber'].unique() #Tous les numéros de groupes sont écrits au bon format 

df_test['Deck'].unique() #Tous les decks sont écrits au même format 

df_test['Side'].unique() #Tous les sides sont écrits au même format 


## C) Détection et traitement des doublons

In [ ]:
print('Il y a',df_train.duplicated().sum(),'doublons dans le df train')
print('Il y a',df_test.duplicated().sum(),'doublons dans le df test')

## D) Détection et traitement des valeurs manquantes

Cette étape du nettoyage des jeux de données est critique pour les algorithmes de Machine Learning. Le traitement des NaNs aura nécessairement une influence sur le résultat final. Il convient donc de prendre les meilleurs hypothèses et d'en explorer plusieurs.

L'objectif est de supprimer un minimum de données car plus on supprime de données, moins l'algorithme de Machine Learning aura de données pour s'entraîner. A contrario, si les NaNs sont remplacés par de mauvaises valeurs, cela bièsera l'algorithme. 

In [ ]:
#Analyse globale
display(pd.DataFrame(df_train.isna().sum(), columns=["Nombre de NA df train"]))
display(pd.DataFrame(df_test.isna().sum(), columns=["Nombre de NA df test"]))
#Oh. My. God.

### 1) Traitement des valeurs manquantes numériques

Dans un premier temps nous allons suivre les hypothèses suivantes :
- Pour l'âge nous allons remplacer les NaNs par la moyenne
- Pour RoomService, FoodCourt, ShoppingMall, Spa et VRDeck nous allons remplacer les NaNs par 0 

Il sera intéressant de faire autrement dans une seconde analyse afin d'observer l'influence de ces paramètres sur l'Accuracy des algorithmes de ML. 

In [ ]:
#On trie le df pour n'avoir que les valeurs numériques 
df_train.select_dtypes(include='number').isna().any()

#DataFrame train

#Age
AgeMean=df_train['Age'].mean().round(0)
df_train['Age']=df_train['Age'].fillna(AgeMean)
#RoomService, FoodCourt, ShoppingMall, Spa et VRDeck
df_train[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']]=df_train[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(0)

#DataFrame test

#Age
AgeMean=df_test['Age'].mean().round(0) #Les moyennes d'age sont les mêmes dans les deux df
df_test['Age']=df_test['Age'].fillna(AgeMean)
#RoomService, FoodCourt, ShoppingMall, Spa et VRDeck
df_test[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']]=df_test[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(0)

### 2) Traitement des valeurs manquantes catégorielles

Dans un premier temps nous allons suivre les hypothèses suivantes. 
Etant donné que les valeurs ont été renseignées de manière ordonnée : 
- Pour HomePlanet, CryoSleep, nous allons chercher une logique avec les proches voisins 
- Pour CryoSleep aller chercher sur internet les distances : Peut etre que ceux qui viennent de plus loin ont décidé de se cryo
- Pour VIP nous allons chercher une corrélation entre les dépenses annexes. En effet, on fait l'hypothèse qu'une personne ayant acheté des services à bord est plus susceptible d'avoir pris une formule VIP. Cependant, il faut garder à l'esprit que cet hypothèse est directement corrélée à l'hypothèse précédente sur les dépenses à bord
- Pour Deck, Num et side nous allons chercher une logique avec les proches voisins. En effet, l'énoncé du concours indique que parfois certains passagers sont en famille. On peut également faire l'hypothèses (qui sera à vérifier en parcourant le dataset) que les passagers de mêmes groupes ont leur cabines à coté. 

In [ ]:
df_train.Deck.value_counts()


In [ ]:
#On trie le df pour n'avoir que les valeurs catégorielles 

#DataFrame train

#Destination
data=df_train.pivot_table(index='HomePlanet', columns='Destination', aggfunc='size', fill_value=0)
data=data.reset_index() #On observe que la plupart des gens vont à TRAPPIST. On décide donc de remplacer les NaNs 
df_train['Destination']=df_train['Destination'].fillna('TRAPPIST')

#HomePlanet 
vip=df_train.pivot_table(index='HomePlanet', columns='VIP', aggfunc='size', fill_value=0)
vip=vip.reset_index() #On remarque qu'aucune personne venant de Earth n'est VIP. 
df_train.loc[(df_train['HomePlanet'].isna()) & (df_train['VIP'] == True)]#On sait donc que ceux-là (291, 365, 405, 7042 et 7786) ne sont pas de Earth
pd.crosstab(df_train.Destination, df_train.HomePlanet, normalize=0) 
#91% des gens qui vont à PSO viennent de Earth, 50% des gens qui vont à Cancri viennent d'Europa et 53% des gens qui vont à Trappist viennent de Earth 
#On met donc en place la boucle suivante pour associer à chaque NaN la HomePlanet la plus probable 
for index, row in df_train.iterrows(): #On parcourt chaque ligne du DataFrame
    if pd.isna(row['HomePlanet']): #On vérifie que la valeur est bien un NaN
        if row['Destination'] == 'CANCRI':
            df_train.at[index, 'HomePlanet'] = 'Europa'
        elif row['Destination'] == 'PSO':
            df_train.at[index, 'HomePlanet'] = 'Earth'
        elif row['Destination'] == 'TRAPPIST':
            df_train.at[index, 'HomePlanet'] = 'Earth'
pd.crosstab(df_train.Destination, df_train.HomePlanet, normalize=0) #On vérifie et OK, on a presque la même distribution 

#VIP
df_train.VIP.value_counts(normalize=True) #On observe que 97% des passagers n'est pas en VIP. On remplace donc par le mode 
df_train['VIP']=df_train['VIP'].fillna(df_train['VIP'].mode()[0])

#CryoSleep 
df_train.CryoSleep.value_counts(normalize=True) #On observe que 65% des passagers on choisi CryoSleep. Pas assez pour remplacer par le mode.  
pd.crosstab(df_train.CryoSleep, df_train.VIP, normalize=1) #89% des VIP ne se cryogénisent pas donc si VIP = cryo non 
for index, row in df_train.iterrows():
    if pd.isna(row['CryoSleep']):
        if row['VIP'] == 'True':
            df_train.at[index, 'CryoSleep'] = False
        else:
            df_train.at[index, 'CryoSleep'] = True

#Deck, Num, Side 
#On observe que Deck, Num et side sont toujours simultanément des NaNs ensemble
#Dans un premier temps je ne parviens pas à observer des liens me permettant de corriger ces NaNs. 
#Pour avancer je supprime ces NaNs en gardant à l'esprit que cela jouera sur l'Accuracy (il constituent 2% du dataset)
df_train=df_train.dropna(axis = 0, how = 'any')


#DataFrame test

#Destination
df_test['Destination']=df_test['Destination'].fillna('TRAPPIST')

#HomePlanet
for index, row in df_test.iterrows():
    if pd.isna(row['HomePlanet']):
        if row['Destination'] == 'CANCRI':
            df_test.at[index, 'HomePlanet'] = 'Europa'
        elif row['Destination'] == 'PSO':
            df_test.at[index, 'HomePlanet'] = 'Earth'
        elif row['Destination'] == 'TRAPPIST':
            df_test.at[index, 'HomePlanet'] = 'Earth'

#VIP
df_test['VIP']=df_test['VIP'].fillna(df_test['VIP'].mode()[0])

#CryoSleep 
for index, row in df_test.iterrows():
    if pd.isna(row['CryoSleep']):
        if row['VIP'] == 'True':
            df_test.at[index, 'CryoSleep'] = False
        else:
            df_test.at[index, 'CryoSleep'] = True

#Deck, Num, Side 
df_test=df_test.dropna(axis = 0, how = 'any')


In [ ]:
#Vérification finale : 
display(pd.DataFrame(df_train.isna().sum(), columns=["Nombre de NA df train"]))
display(pd.DataFrame(df_test.isna().sum(), columns=["Nombre de NA df test"]))

## E) Deuxieme passe de traitement de la cohérence des données 

Certaines variabes n'ont pas pu être traitées avant le traitement des NaNs car ils en contenaient

In [ ]:
#DataFrame Train
df_train['CryoSleep']=df_train['CryoSleep'].astype(bool)
df_train[['Num','FoodCourt','Spa','RoomService','ShoppingMall','VRDeck','Age']]=df_train[['Num','FoodCourt','Spa','RoomService','ShoppingMall','VRDeck','Age']].astype(int)


#Age a mettre en int
#Room service a mettre en int (vérifier l'explication sur le site)

#DataFrame Test
df_test['CryoSleep']=df_test['CryoSleep'].astype(bool)
df_test[['Num','FoodCourt','Spa','RoomService','ShoppingMall','VRDeck','Age']]=df_test[['Num','FoodCourt','Spa','RoomService','ShoppingMall','VRDeck','Age']].astype(int)


## F) Détection et traitement des Outliers
Ce traitement ne concerne que les variables quantitatives

In [ ]:
num=df_train.select_dtypes(include='number')
columns = num.columns

plt.figure(figsize=(20, 15)) 
for i, col in enumerate(columns, 1):
    plt.subplot(3, 3, i)
    sns.boxplot(y=num[col])
    plt.title(f'Boxplot de {col}')
plt.tight_layout()
plt.show()

#On observe qu'il y a des outliers mais qui ne constituent pas des valeurs aberrantes. 

In [ ]:
num2=df_test.select_dtypes(include='number')
columns = num2.columns

plt.figure(figsize=(20, 15)) 
for i, col in enumerate(columns, 1):
    plt.subplot(3, 3, i)
    sns.boxplot(y=num2[col])
    plt.title(f'Boxplot de {col}')
plt.tight_layout()
plt.show() 

#Idem que pour le précédent DataSet, on observe qu'il y a des outliers mais qui ne constituent pas des valeurs aberrantes. 

In [ ]:
#Etude de la distribution des âges 
plt.figure(figsize=(20, 8))

plt.subplot(1, 2, 1)
sns.histplot(df_train['Age'], bins=12, kde=True, color='red')
plt.title("Distribution de l'âge dans le df train")

plt.subplot(1, 2, 2)
sns.histplot(df_test['Age'], bins=12, kde=True, color='blue')
plt.title("Distribution de l'âge dans le df test")

plt.tight_layout()
plt.show()

# II) Machine Learning

In [ ]:
#L'idée est pour le sport de tester plusieurs algos de ML de classification et de pousser chaque étude au max en variant leurs hyperparamètres (GridSearchCV)
#A la fin on fera un tableau récapitulatif montrant les meilleurs scores de chacun pour permettre de décider duquel choisir. 
#Il me semble que dans le premier notebook, il y avait une boucle permettant de faire ça.  

NOTE : On veillera à regarder quelles sont les variables qui influent le plus sur ces algos de ML. Cela permettra de jouer sur leur étape de traitement des valeurs manquantes pour voir ce que ça fait vis a vis de l'accuracy

## A) Régression logistique
### 1) Préparation et modélisation des données

In [ ]:
#On copie les DataFrames pour effectuer des modifications dessus
df_train_RL=df_train.copy()
df_test_RL=df_test.copy()

Discrétisation des variables

In [ ]:
#Age
df_train_RL['Age']=pd.cut(x=df_train_RL['Age'], bins=[0,10,20,30,40,50,60,70,80], labels=['0-10','10-20','20-30','30-40','40-50','50-60','60-70','70-80'], include_lowest= True)
df_test_RL['Age']=pd.cut(x=df_test_RL['Age'], bins=[0,10,20,30,40,50,60,70,80], labels=['0-10','10-20','20-30','30-40','40-50','50-60','60-70','70-80'], include_lowest= True)


Séparation des variables explicatives de la variable à prédire

In [ ]:
data = df_train_RL.drop('Transported', axis=1)
target = df_train_RL['Transported']

Séparation des données d'entraînement et de test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.2, random_state=66)

Dichotomisation des variables

La dichotomisation post train-test split prévient les fuites de données en évitant que des informations de l'ensemble de test n'influencent involontairement le modèle. En la réalisant après la division, elle assure également que le modèle est entraîné de manière à simuler un environnement réel, capable de gérer des catégories inconnues rencontrées dans de nouvelles données.


In [ ]:
#L'objectif est de ne plus avoir de variables qualitatives. En effet, certains modèles de ML sont incapables d'interpréter des variables qualitatives

#DataFrame Train

enc=preprocessing.OneHotEncoder(handle_unknown='ignore')
X_train_enc=enc.fit_transform(X_train)
X_test_enc=enc.transform(X_test)

#DataFrame Test

df_test_RL_enc=enc.transform(df_test_RL)

### 2) Premier modèle de régression logistique

In [ ]:
#Construction du classifieur 
clf=linear_model.LogisticRegression(C=1.0, max_iter=1000)
#Entrainement de l'algorithme sur l'ensemble d'entrainement
clf.fit(X_train_enc, y_train)

In [ ]:
#Evaluation du modèle
y_pred=clf.predict(X_test_enc)
cm = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
cm


In [ ]:
#Accuracy
accuracy = clf.score(X_train_enc, y_train)
print("L'accuracy du modèle d'entrainement est",accuracy )
accuracy = clf.score(X_test_enc, y_test)
print("L'accuracy du modèle de test est",accuracy )
print("Il y a énormément d'overfitting")

In [ ]:
#Modification du seuil de précision
probs = clf.predict_proba(X_test_enc)
y_preds = np.where(probs[:,1]>0.4,1,0)
cm = pd.crosstab(y_test, y_preds, rownames=['Classe réelle'], colnames=['Classe prédite'])
cm #Toujours pas foufou

Ce premier modèle de classification est mauvais. En effet, la différence d'accuracy est énorme entre le modèle d'entrainement et le modèle de test. Un telle différence implique un phénomène de sur-apprentissage (overfitting). Il faut à tout prix éviter cela.

Il convient donc d'étudier le réglage des hyperparamètres du modèle. 

### 3) Affinage du modèle et réglage des hyperparamètres

In [ ]:
#Création d'un dictionnaire comprenant les valeurs possibles pour les hyperparamètres 
parametres = {
    'C': np.arange(0.1, 1, 0.1),
    'penalty': ['l2', 'l1', 'elasticnet'],
    'solver': ['liblinear', 'saga'], 
    'max_iter': [1000],
}

In [ ]:
grid_regLog = model_selection.GridSearchCV(estimator=clf, param_grid=parametres, cv=3, n_jobs=-1) #Application de la fonction au classifieur
grille = grid_regLog.fit(X_train_enc,y_train) #Entrainement sur l'ensemble d'entrainement
print(pd.DataFrame.from_dict(grille.cv_results_).loc[:,['params', 'mean_test_score']]) #Fonctionne en 15secs

In [ ]:
print(grid_regLog.best_params_)

In [ ]:
y_pred=grid_regLog.predict(X_test_enc)
pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])


In [ ]:
accuracy_train_RL = grid_regLog.score(X_train_enc, y_train)
print("L'accuracy du modèle d'entrainement est",accuracy_train_RL )
accuracy_test_RL = grid_regLog.score(X_test_enc, y_test)
print("L'accuracy du modèle de test est",accuracy_test_RL )

In [ ]:
# Impression du rapport de classification 
print(classification_report(y_test, y_pred)) 

L'overfitting est considérablement réduit, mais n'est toujours pas satisfaisant. Le score global est trop faible.
Le modèle de régression linéaire................................................................................................ dire qu'on s'y attendait

## B) KNN (K Nearest Neighbors)
### 1) Préparation et modélisation des données

In [ ]:
#On copie les DataFrames pour effectuer des modifications dessus
df_train_KNN=df_train.copy()
df_test_KNN=df_test.copy()

Séparation des variables explicatives de la variable à prédire

In [ ]:
data=df_train_KNN.drop('Transported', axis=1)
target=df_train_KNN['Transported']
target =  [1 if x==True else 0 for x in target]
print(target)

Transformation des variables catégorielles en variables binaires

In [ ]:
data_matrix=pd.get_dummies(data, dtype=float)
data_matrix.head(1)

Séparation des données d'entraînement et de test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_matrix, target, test_size = 0.2, random_state=66)

Dichotomisation des variables

In [ ]:
scaler=preprocessing.StandardScaler().fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

### 2) Premier modèle de classification

In [ ]:
#Construction du classifieur 
knn = neighbors.KNeighborsClassifier(n_neighbors=7, metric='minkowski')

In [ ]:
#Ajustement de l'algorithme sur l'ensemble d'entrainement
knn.fit(X_train_scaled, y_train)

In [ ]:
#Evaluation du modèle
y_pred = knn.predict(X_test_scaled)
cm=pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
cm

In [ ]:
#Accuracy
accuracy = knn.score(X_train_scaled, y_train)
print("L'accuracy du modèle d'entrainement est",accuracy )
accuracy = knn.score(X_test_scaled, y_test)
print("L'accuracy du modèle de test est",accuracy )
print("On observe qu'il y a tout de même un peu d'overfitting")

### 3) Affinage du modèle et réglage des hyperparamètres

In [ ]:
#Création d'un dictionnaire comprenant les valeurs possibles pour les hyperparamètres 
parametres = {'n_neighbors' : np.arange(2, 16, 1),
              'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'metric' : ['manhattan', 'chebyshev', 'minkowski']
}

In [ ]:
grid_knn = model_selection.GridSearchCV(estimator=knn, param_grid=parametres, cv=5) #Application de la fonction au classifieur
grille = grid_knn.fit(X_train_scaled,y_train) #Entrainement sur l'ensemble d'entrainement
print(pd.DataFrame.from_dict(grille.cv_results_).loc[:,['params', 'mean_test_score']]) #Fonctionne en 2mins17

In [ ]:
print(grid_knn.best_params_)

In [ ]:
y_pred=grid_knn.predict(X_test_scaled)
pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

In [ ]:
accuracy_train_knn = grid_knn.score(X_train_scaled, y_train)
print("L'accuracy du modèle d'entrainement est",accuracy_train_knn )
accuracy_test_knn = grid_knn.score(X_test_scaled, y_test)
print("L'accuracy du modèle de test est",accuracy_test_knn )
#On a réduit l'overfitting mais on a également réduit la précision du modèle

In [ ]:
# Impression du rapport de classification 
print(classification_report(y_test, y_pred)) 

## C) Support Vector Machine (SVM)
### 1) Préparation des données

In [ ]:
#On copie les DataFrames pour effectuer des modifications dessus
df_train_SVM=df_train.copy()
df_test_SVM=df_test.copy()

Séparation des variables explicatives de la variable à prédire

In [ ]:
data=df_train_SVM.drop('Transported', axis=1)
target=df_train_SVM['Transported']
target =  [1 if x==True else 0 for x in target]
print(target)

Transformation des variables catégorielles en variables binaires

In [ ]:
data_matrix=pd.get_dummies(data, dtype=float)
data_matrix.head(1)

Séparation des données d'entraînement et de test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_matrix, target, test_size = 0.2, random_state=66)

Dichotomisation des variables

In [ ]:
scaler=preprocessing.StandardScaler().fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

### 2) Premier modèle

In [ ]:
#Construction du classifieur 
clf_svm=svm.SVC(gamma=0.01, kernel = 'poly')
#Evaluation du modèle
clf_svm.fit(X_train_scaled,y_train)

In [ ]:
#Evaluation du modèle
y_pred=clf_svm.predict(X_test_scaled)
pd.crosstab(y_test,y_pred,rownames=['Classe réelle'], colnames=['Classe prédite'])

In [ ]:
#Accuracy
accuracy = clf_svm.score(X_train_scaled, y_train)
print("L'accuracy du modèle d'entrainement est",accuracy )
accuracy = clf_svm.score(X_test_scaled, y_test)
print("L'accuracy du modèle de test est",accuracy )
print("Le score est intérressant car il n'y a pas d'overfitting !")

### 3) Affinage du modèle et réglage des hyperparamètres

In [ ]:
parametres = {'C':[0.1,1,10], 
              'kernel':['rbf','linear','poly'], 
              'gamma':[0.001, 0.1, 1]
              }

In [ ]:
grid_svm = model_selection.GridSearchCV(estimator=clf_svm, param_grid=parametres, cv=5) #Application de la fonction au classifieur
grille = grid_svm.fit(X_train_scaled,y_train) #Entrainement sur l'ensemble d'entrainement
print(pd.DataFrame.from_dict(grille.cv_results_).loc[:,['params', 'mean_test_score']]) #Fonctionne en mins

In [ ]:
print(grid_svm.best_params_)

In [ ]:
y_pred=grid_svm.predict(X_test_scaled)
pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

In [ ]:
accuracy_train_svm = grid_knn.score(X_train_scaled, y_train)
print("L'accuracy du modèle d'entrainement est",accuracy_train_svm )
accuracy_test_svm = grid_knn.score(X_test_scaled, y_test)
print("L'accuracy du modèle de test est",accuracy_test_svm )

In [ ]:
# Impression du rapport de classification 
print(classification_report(y_test, y_pred)) 

ATTENTION IL FAUDRA AUSSI APPLIQUER LES MEMES PREPROCESSING AU DF TEST 

FAIRE UN TABLEAU RECAPITULATIF DE TOUTES LES ACCURACY POUR LES COMPARER

Aide pour NaNs : https://www.kaggle.com/competitions/spaceship-titanic/discussion/315987